In [1]:
import os
import shutil
import subprocess
import sys
import pandas as pd
import numpy as np

# Setup paths
EXPERIMENT_DIR = '/home/code/experiments/001_baseline'
DATA_DIR = '/home/data'
SUBMISSION_PATH = '/home/submission/submission.csv'

# Copy sample submission to experiment dir
shutil.copy(os.path.join(DATA_DIR, 'sample_submission.csv'), os.path.join(EXPERIMENT_DIR, 'submission.csv'))

print("Setup complete.")

Setup complete.


In [2]:
# Compile C++ code
cpp_files = ['tree_packer_v21.cpp', 'bp.cpp']
executables = ['tree_packer_v21.exe', 'bp.exe']

compile_cmds = [
    f"g++ -O3 -march=native -std=c++17 -fopenmp -o {os.path.join(EXPERIMENT_DIR, 'tree_packer_v21.exe')} {os.path.join(EXPERIMENT_DIR, 'tree_packer_v21.cpp')}",
    f"g++ -O3 -std=c++17 -o {os.path.join(EXPERIMENT_DIR, 'bp.exe')} {os.path.join(EXPERIMENT_DIR, 'bp.cpp')}"
]

for cmd in compile_cmds:
    print(f"Compiling: {cmd}")
    subprocess.check_call(cmd, shell=True)

print("Compilation complete.")

Compiling: g++ -O3 -march=native -std=c++17 -fopenmp -o /home/code/experiments/001_baseline/tree_packer_v21.exe /home/code/experiments/001_baseline/tree_packer_v21.cpp


Compiling: g++ -O3 -std=c++17 -o /home/code/experiments/001_baseline/bp.exe /home/code/experiments/001_baseline/bp.cpp


Compilation complete.


In [3]:
class SantaSolver:
    def __init__(self, experiment_dir, input_file='submission.csv'):
        self.experiment_dir = experiment_dir
        self.input_file = os.path.join(experiment_dir, input_file)
        self.output_file = os.path.join(experiment_dir, 'submission_v21.csv')
        self.packer_exe = os.path.join(experiment_dir, 'tree_packer_v21.exe')
        self.bp_exe = os.path.join(experiment_dir, 'bp.exe')
        
    def optimize(self, iters=1000, restarts=4):
        cmd = f"{self.packer_exe} -i {self.input_file} -o {self.output_file} -n {iters} -r {restarts}"
        print(f"Running: {cmd}")
        
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            shell=True
        )
        
        final_score = None
        for line in process.stdout:
            print(line, end='')
            if "Final:" in line:
                try:
                    final_score = float(line.split()[1])
                except:
                    pass
        
        process.wait()
        if process.returncode != 0:
            raise RuntimeError("Optimization failed")
            
        # Update input file for next step
        if os.path.exists(self.output_file):
            shutil.copy(self.output_file, self.input_file)
            
        return final_score

    def back_propagate(self):
        cmd = f"{self.bp_exe} {self.input_file} {self.input_file}"
        print(f"Running: {cmd}")
        
        process = subprocess.Popen(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            shell=True
        )
        
        final_score = None
        for line in process.stdout:
            print(line, end='')
            if "Final score:" in line:
                try:
                    final_score = float(line.split(":")[1])
                except:
                    pass
        
        process.wait()
        if process.returncode != 0:
            raise RuntimeError("Backward propagation failed")
            
        return final_score

print("SantaSolver class defined.")

SantaSolver class defined.


In [4]:
# Run optimization
solver = SantaSolver(EXPERIMENT_DIR)

# Run a few iterations
# For baseline, we'll do 1 pass of optimization and 1 pass of BP
print("Starting optimization...")
score1 = solver.optimize(iters=1000, restarts=4)
print(f"Score after optimization: {score1}")

print("Starting backward propagation...")
score2 = solver.back_propagate()
print(f"Score after BP: {score2}")

final_score = score2 if score2 is not None else score1
print(f"Final Score: {final_score}")

Starting optimization...
Running: /home/code/experiments/001_baseline/tree_packer_v21.exe -i /home/code/experiments/001_baseline/submission.csv -o /home/code/experiments/001_baseline/submission_v21.csv -n 1000 -r 4
Tree Packer v21 - ENHANCED (26 threads)
NEW: Swap moves, multi-angle restarts, higher SA temperature
Iterations: 1000, Restarts: 4
Processing all n=1..200 concurrently
Loading /home/code/experiments/001_baseline/submission.csv...
Loaded 200 configs
Initial: 173.652299

Phase 1: Parallel optimization...

n=  1: 1.000000 -> 0.661267 (33.8733%)


n=  2: 0.733589 -> 0.450785 (38.5508%)


n=  3: 0.930301 -> 0.544534 (41.4669%)


n=  4: 1.039642 -> 0.496133 (52.2785%)


n=  5: 0.900336 -> 0.533596 (40.7337%)
n=  9: 1.315697 -> 0.616863 (53.1151%)


...

n= 19: 0.798290 -> 0.621124 (from n=20 removal, 22.1932%)
n= 18: 0.741692 -> 0.652423 (from n=19 removal, 12.0358%)


n= 17: 0.756711 -> 0.662793 (from n=18 removal, 12.4114%)
n= 16: 0.832550 -> 0.689366 (from n=17 removal, 17.1982%)
n= 15: 0.884838 -> 0.709150 (from n=16 removal, 19.8554%)


n= 14: 0.867471 -> 0.738432 (from n=15 removal, 14.8752%)


n= 13: 0.837615 -> 0.603667 (from n=14 removal, 27.9303%)
n= 12: 0.788261 -> 0.638865 (from n=13 removal, 18.9527%)
Pass 1 complete, continuing...

Initial: 173.652299
Final:   131.339117
Improve: 42.313182 (24.3666%)
Phase 1 improved: 200 configs
Phase 2 back-prop improved: 184 configs
Time:    737.5s (with 26 threads)
Saved /home/code/experiments/001_baseline/submission_v21.csv
Score after optimization: 131.339117
Starting backward propagation...
Running: /home/code/experiments/001_baseline/bp.exe /home/code/experiments/001_baseline/submission.csv /home/code/experiments/001_baseline/submission.csv


Backward Propagation Optimizer
Loading /home/code/experiments/001_baseline/submission.csv...
Loaded 200 configurations
Starting Backward Propagation...
Initial score: 131.33911665

improved 174 from n=200 10.89766423 -> 10.87371571
improved 173 from n=200 10.89502907 -> 10.86234034
improved 101 from n=200 8.13137430 -> 8.09856871
improved 100 from n=200 8.11959799 -> 8.09278059
improved 96 from n=200 8.00480076 -> 7.97544859
improved 42 from n=200 5.36830787 -> 5.31688563
improved 41 from n=200 5.30650145 -> 5.26801427
improved 40 from n=200 5.25597615 -> 5.24143086
improved 39 from n=200 5.20284253 -> 5.20040037
improved 35 from n=200 4.97269040 -> 4.94504562
improved 34 from n=200 4.89416185 -> 4.80885861
improved 33 from n=200 4.89290807 -> 4.79281530
improved 32 from n=200 4.74308856 -> 4.72855396
improved 25 from n=200 4.16799609 -> 4.14834751
improved 24 from n=200 4.15605055 -> 4.10254367
improved 36 from n=198 5.01690654 -> 5.01228587
improved 35 from n=198 4.94504562 -> 4.9013

In [5]:
# Generate submission file
# The solver updates 'submission.csv' in the experiment dir
# We need to copy it to the submission path
if os.path.exists(os.path.join(EXPERIMENT_DIR, 'submission.csv')):
    os.makedirs(os.path.dirname(SUBMISSION_PATH), exist_ok=True)
    shutil.copy(os.path.join(EXPERIMENT_DIR, 'submission.csv'), SUBMISSION_PATH)
    print(f"Submission saved to {SUBMISSION_PATH}")
else:
    print("Error: submission.csv not found!")

# Verify submission format
df = pd.read_csv(SUBMISSION_PATH)
print(df.head())
print(df.columns)

Submission saved to /home/submission/submission.csv
      id                      x                      y                     deg
0  001_0  s-3.29456262562994921  s-3.26517018860202250  s225.00186776555784758
1  002_0   s0.39389830954297391   s0.62786420194283746   s66.37237360073846150
2  002_1  s-0.12902527323512322   s0.31967287859283390  s246.37236770793217351
3  003_0   s0.29207599944320522  s-0.72470967445076635   s99.52799071106374007
4  003_1   s0.04471255415282656  s-0.11614895861227405  s132.94539848560790141
Index(['id', 'x', 'y', 'deg'], dtype='object')
